In [1]:
import numpy as np
import pandas as pd
import pyxdf

# mne imports
import mne
from mne import io
from mne.datasets import sample

# EEGNet-specific imports
from EEGModels import EEGNet
import tensorflow
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# PyRiemann imports
from pyriemann.estimation import XdawnCovariances
from pyriemann.tangentspace import TangentSpace

#Sklearn imports
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.utils import compute_class_weight
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# tools for plotting confusion matrices
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from pyriemann.utils.viz import plot_confusion_matrix

In [2]:
#Loading EEG data
samples = 1233

X_train_01 = np.loadtxt("Full_data_X_P01_5.csv")
X_train_02 = np.loadtxt("Full_data_X_P02_5.csv")
X_train_03 = np.loadtxt("Full_data_X_P03_5.csv")
X_train_04 = np.loadtxt("Full_data_X_P04_5.csv")
X_train_05 = np.loadtxt("Full_data_X_P05_5.csv")
X_train_06 = np.loadtxt("Full_data_X_P06_5.csv")
X_train_07 = np.loadtxt("Full_data_X_P07_5.csv")
X_train_08 = np.loadtxt("Full_data_X_P08_5.csv")
X_train_09 = np.loadtxt("Full_data_X_P09_5.csv")
X_train_10 = np.loadtxt("Full_data_X_P10_5.csv")
X_train_11 = np.loadtxt("Full_data_X_P11_5.csv")
X_train_12 = np.loadtxt("Full_data_X_P12_5.csv")
X_train_13 = np.loadtxt("Full_data_X_P13_5.csv")
X_train_14 = np.loadtxt("Full_data_X_P14_5.csv")
X_train_15 = np.loadtxt("Full_data_X_P15_5.csv")
X_train_16 = np.loadtxt("Full_data_X_P16_5.csv")
X_train_17 = np.loadtxt("Full_data_X_P17_5.csv")
X_train_18 = np.loadtxt("Full_data_X_P18_5.csv")
X_train_19 = np.loadtxt("Full_data_X_P19_5.csv")
X_train_20 = np.loadtxt("Full_data_X_P20_5.csv")

#Reshaping
X_train_01 = X_train_01.reshape(
     X_train_01.shape[0], X_train_01.shape[1] // samples, samples)

X_train_02 = X_train_02.reshape(
     X_train_02.shape[0], X_train_02.shape[1] // samples, samples)

X_train_03 = X_train_03.reshape(
     X_train_03.shape[0], X_train_03.shape[1] // samples, samples)

X_train_04 = X_train_04.reshape(
     X_train_04.shape[0], X_train_04.shape[1] // samples, samples)

X_train_05 = X_train_05.reshape(
     X_train_05.shape[0], X_train_05.shape[1] // samples, samples)

X_train_06 = X_train_06.reshape(
     X_train_06.shape[0], X_train_06.shape[1] // samples, samples)

X_train_07 = X_train_07.reshape(
     X_train_07.shape[0], X_train_07.shape[1] // samples, samples)

X_train_08 = X_train_08.reshape(
     X_train_08.shape[0], X_train_08.shape[1] // samples, samples)

X_train_09 = X_train_09.reshape(
     X_train_09.shape[0], X_train_09.shape[1] // samples, samples)

X_train_10 = X_train_10.reshape(
     X_train_10.shape[0], X_train_10.shape[1] // samples, samples)

X_train_11 = X_train_11.reshape(
     X_train_11.shape[0], X_train_11.shape[1] // samples, samples)

X_train_12 = X_train_12.reshape(
     X_train_12.shape[0], X_train_12.shape[1] // samples, samples)

X_train_13 = X_train_13.reshape(
     X_train_13.shape[0], X_train_13.shape[1] // samples, samples)

X_train_14 = X_train_14.reshape(
     X_train_14.shape[0], X_train_14.shape[1] // samples, samples)

X_train_15 = X_train_15.reshape(
     X_train_15.shape[0], X_train_15.shape[1] // samples, samples)

X_train_16 = X_train_16.reshape(
     X_train_16.shape[0], X_train_16.shape[1] // samples, samples)

X_train_17 = X_train_17.reshape(
     X_train_17.shape[0], X_train_17.shape[1] // samples, samples)

X_train_18 = X_train_18.reshape(
     X_train_18.shape[0], X_train_18.shape[1] // samples, samples)

X_train_19 = X_train_19.reshape(
     X_train_19.shape[0], X_train_19.shape[1] // samples, samples)

X_train_20 = X_train_20.reshape(
     X_train_20.shape[0], X_train_20.shape[1] // samples, samples)

In [ ]:
#Choosing which participants should be in the training data
X_training = np.vstack([X_train_20, X_train_19, X_train_18, X_train_17, X_train_16, X_train_15, X_train_14, X_train_13, X_train_12, X_train_11, X_train_10, X_train_09, X_train_08, 
                        X_train_07, X_train_06, X_train_05, X_train_04, X_train_03, X_train_02])

In [22]:
#Loading labels
Y_train_01 = np.loadtxt("Full_data_Y_P01_5.csv")
Y_train_02 = np.loadtxt("Full_data_Y_P02_5.csv")
Y_train_03 = np.loadtxt("Full_data_Y_P03_5.csv")
Y_train_04 = np.loadtxt("Full_data_Y_P04_5.csv")
Y_train_05 = np.loadtxt("Full_data_Y_P05_5.csv")
Y_train_06 = np.loadtxt("Full_data_Y_P06_5.csv")
Y_train_07 = np.loadtxt("Full_data_Y_P07_5.csv")
Y_train_08 = np.loadtxt("Full_data_Y_P08_5.csv")
Y_train_09 = np.loadtxt("Full_data_Y_P09_5.csv")
Y_train_10 = np.loadtxt("Full_data_Y_P10_5.csv")
Y_train_11 = np.loadtxt("Full_data_Y_P11_5.csv")
Y_train_12 = np.loadtxt("Full_data_Y_P12_5.csv")
Y_train_13 = np.loadtxt("Full_data_Y_P13_5.csv")
Y_train_14 = np.loadtxt("Full_data_Y_P14_5.csv")
Y_train_15 = np.loadtxt("Full_data_Y_P15_5.csv")
Y_train_16 = np.loadtxt("Full_data_Y_P16_5.csv")
Y_train_17 = np.loadtxt("Full_data_Y_P17_5.csv")
Y_train_18 = np.loadtxt("Full_data_Y_P18_5.csv")
Y_train_19 = np.loadtxt("Full_data_Y_P19_5.csv")
Y_train_20 = np.loadtxt("Full_data_Y_P20_5.csv")

In [ ]:
#Choosing which participants should be in the training data
Y_training = np.hstack([Y_train_20, Y_train_19, Y_train_18, Y_train_17, Y_train_16, Y_train_15, Y_train_14, Y_train_13, Y_train_12, Y_train_11, Y_train_10, Y_train_09, Y_train_08, 
                        Y_train_07, Y_train_06, Y_train_05, Y_train_04, Y_train_03, Y_train_02])

In [ ]:
#Choosing which participants should be in the test data
X_testing = np.vstack([X_train_01])
Y_testing = np.hstack([Y_train_01])

In [25]:
kernels, chans, samples = 1, 16, 1233

model = EEGNet(nb_classes = 1, Chans = chans, Samples = samples, 
               dropoutRate = 0.25, kernLength = 125, F1 = 8, D = 2, F2 = 16, 
               dropoutType = 'Dropout')

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001),
              metrics = ['accuracy'])

In [26]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_training.reshape(X_training.shape[0], -1)).reshape(X_training.shape)
X_test = scaler.fit_transform(X_testing.reshape(X_testing.shape[0], -1)).reshape(X_testing.shape)

randomize_train = np.arange(len(X_train))
np.random.shuffle(randomize_train)
X_train = X_train[randomize_train]
Y_train = Y_training[randomize_train]

randomize_test = np.arange(len(X_test))
np.random.shuffle(randomize_train)
X_test = X_test[randomize_test]
Y_test = Y_testing[randomize_test]

In [27]:
X_train  = X_train.reshape(X_train.shape[0], chans, samples, kernels)
X_test  = X_test.reshape(X_test.shape[0], chans, samples, kernels)

In [ ]:
fittedModel = model.fit(X_train, Y_train, batch_size = 1024, epochs = 20)

In [ ]:
score = model.evaluate(X_test, Y_test)

pred_val = model.predict(X_test)
#print("Pred val: ", pred_val)
pred_val = np.where(pred_val > 0.5, 1, 0)

labels = ["Non-Alcohol", "Alcohol"]
cm = confusion_matrix(Y_test, pred_val)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(cmap=plt.cm.Blues)
plt.show()

tn = cm[0][0] #true negatives
fn = cm[0][1] #false positives

accuracy = score[1]
F1_score = f1_score(y_true=Y_test, y_pred=pred_val)
precision= precision_score(y_true=Y_test, y_pred=pred_val)
recall = recall_score(y_true=Y_test, y_pred=pred_val)
specificity = tn/(tn+fn)

print("The accuracy of the model with cross validation is", accuracy)
print("The precision score of the model with cross validation is", precision)
print("The recall score of the model with cross validation is", recall)
print("The F1 score of the model with cross validation is", F1_score)
print("The specificity score of the model with cross validation is", specificity)